In [46]:
from CheckEnv import check_env
from CheckPlayer import check_agent
from setup import path, SHOT_PATH, time_run_game
import pandas as pd
import importlib.util
import numpy as np
import sys
import json

SHOT_PATH = 'A:/AutoTrain/GAME/'

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHOT_PATH}base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

def load_module_player(player):
    return  importlib.util.spec_from_file_location('Agent_player', f"{SHOT_PATH}Agent/{player}/Agent_player.py").loader.load_module()

In [47]:
df_run = pd.read_json(f'{SHOT_PATH}Log/State.json')
df_agent = pd.read_json(f'{SHOT_PATH}Log/StateAgent.json')
df_env = pd.read_json(f'{SHOT_PATH}Log/StateEnv.json')

list_tong = list(df_run.index)
dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
dict_agent = json.load(open(f'{SHOT_PATH}Log/agent_all.json'))
PASS_LEVEL = 250

In [45]:
def save_json(path_save_json, dict_save):
    with open(path_save_json, 'w') as f:
        json.dump(dict_save, f)
save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)
# dict_level.to


In [50]:
dict_level

{'ngoxuanphong_1672396681': {'Can_Split_Level': 'True',
  'level_max': 2,
  'id_remove': [10,
   11,
   3,
   8,
   4,
   11,
   5,
   2,
   7,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21],
  '1': [[3, 8, 4],
   [9147.0, 9370.0, 9452.0],
   ['AgentBias4', 'AgentBias9', 'AgentBias5']],
  '2': [[5, 2, 7],
   [9478.0, 9520.0, 9714.0],
   ['AgentBias6', 'AgentBias3', 'AgentBias8']]},
 'ngoxuanphong_1672398548': {'Can_Split_Level': 'True',
  'level_max': 1,
  'id_remove': [10,
   11,
   8,
   5,
   2,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21],
  '1': [[8, 5, 2],
   [7471.0, 9311.0, 9542.0],
   ['AgentBias9', 'AgentBias6', 'AgentBias3']]}}

In [72]:
from server.mysql_connector import get_db_cursor
mycursor, mydb = get_db_cursor()
mycursor.execute(f"SELECT * FROM CodeBot WHERE NotificateID = '100'")

data_env_upload = mycursor.fetchall()
data_env_upload

[(6,
  2,
  'ngoxuanphong_1672567938',
  datetime.datetime(2023, 1, 1, 10, 12, 32),
  100,
  0),
 (10,
  2,
  'ngoxuanphong_1672569908',
  datetime.datetime(2023, 1, 1, 10, 45, 20),
  100,
  0),
 (13,
  2,
  'ngoxuanphong_1672570588',
  datetime.datetime(2023, 1, 1, 10, 56, 40),
  100,
  0)]

In [37]:
df_run['ngoxuanphong_1672396681'][0] = 0

In [38]:
df_run.to_json(f'{SHOT_PATH}Log/State.json')

In [189]:
df_run['Splendor_v3'].iloc[0] = 5108.0
df_run['Splendor_v3'].iloc[1] = 5530.0
df_run['Splendor_v3'].iloc[3] = 5160.0

df_run['Splendor_v3'].iloc[4] = 5536.0
df_run['Splendor_v3'].iloc[5] = 5592.0
df_run['Splendor_v3'].iloc[7] = 5535.0

df_run.to_json(f'{SHOT_PATH}Log/State.json')

In [168]:
df_run.iloc[:, 7:]

,Splendor_v3,MachiKoro,Splendor_v2
0,DONE,7533.0,9328.0
1,DONE,8233.0,9814.0
2,5690.0,7990.0,9711.0
3,DONE,8072.0,9579.0
4,5536.0,7646.0,9616.0
5,5592.0,8868.0,9310.0
6,5705.0,7688.0,9422.0
7,5535.0,8353.0,9595.0
8,5649.0,8128.0,9811.0
9,5639.0,8532.0,9613.0


In [126]:
df_env['CHECK'] = np.nan
df_env['NOTE'] = np.nan
df_env.to_json(f'{SHOT_PATH}Log/StateEnv.json')

In [123]:
df_env['CHECK'] = np.nan
df_env['NOTE'] = np.nan
df_env.to_json(f'{SHOT_PATH}Log/StateEnv.json')

# Suawr laji maays file linh tinh vo van

In [82]:
# df_run = df_run.drop(labels=['TLMN', 'TLMN_v2'], axis=1)
# df_run = df_run.drop(11)
# df_run['Splendor_v3'] = np.nan
# df_run['Splendor_v2'] = np.nan
# df_run['MachiKoro'] = np.nan
# df_run.to_json(f'{SHOT_PATH}Log/State.json')